In [7]:
# from flask_other.app import app
from flask import Flask
import os,sys
# sys.path.append('/home/kai/data/wei/Chatbot1.0/')
sys.path.append('../')
# sys.path.append('/home/kai/data/Chatbot1.0/API/v1/frontend/')
app=Flask("demo")
from flask import  render_template,request
from app.mong_database import MongoManager
import time
import re
mongo_db=MongoManager(server_ip='chatbotdb')

import urllib.request
from urllib.parse import quote
import pandas as pd
import numpy as np
import time
import re
import requests

##################################
url = 'http://localhost:8889/chatbotv1'


def get_data(sentence):
    
    response = requests.get(url, timeout=10, params={'data':sentence, 
                                                     'action':'chat', 
                                                     'sessionId':cache.sessionId})
    
    if response.status_code == 200:
        req = response.json()
        if req.get('status').lower() == 'successful':
            msg = req.get('message')
        else:
            msg = req

        return msg
    else:
        return 'Internet Connection Issue, error code {}'.format(response.status_code)
def get_data(sentence):
    
    return 'data get'


@app.route('/')
@app.route('/index')
def index0():
    return render_template(
                           # "index0.html",
                           "index.html",
                           )
@app.route("/predict", methods= ["POST"])
def background_process():
    if request.method == 'POST':
        try:
            query = request.form.get('query')#前端查询的内容
            if query:

                    print('query',query)
                    # time.sleep(5)
                    result = get_data(query)
                    print('result',result)
                    mongo_db.save_query(query, str(result))
                    print('saving to mongo db successfully!')
                    return str(result)
                    

            else:

                    return str('请输入查询内容')


        except Exception as e:
            print(e)

            if 'duplicate' in str(e):
                e_str = e.details['errmsg']
                dup_id=re.search('\{ : "(.*)" \}',e_str).group(1)
                print('重复查询同一句话,存储时使用相同的_id_',dup_id)
                mongo_db.update_dup_query( dup_id, str(result))
                return str(result)

            else:
                print(e)
                print('有问题，MM出故障啦')
                return str('MM出故障啦')

        # finally:
        #     # print(e)
        #     print('有问题，MM出故障啦。。')
        #     return str('MM出故障啦。。')

    else:
        return 'ok'
    
    
@app.route("/newConversation", methods= ["POST"])
def new_conversation():
    cache.new_conversation()
    print('new conversation called!!!!!!!!!!!!!!!!!!!!!!!')

class Cache:
    def __init__(self):
        self.new_conversation()
            
    def new_conversation(self):
        req = requests.get(url, timeout=10, params={'action':'create'})
        if req.status_code == 200:
            msg = req.json()
            if msg['status'] == 'successful':
                self.sessionId = msg['message']['sessionId']
                print(self.sessionId)
            else:
                print(req.content)
                raise ValueError('cannot get new sessionId, cannot start chat')
        else:
            print(req.content)


        
# cache = Cache()   




chatbotdb


In [ ]:
app.debug = False
port = 6006
print('running at http://10.0.24.31:{}'.format(port))
app.run(host='0.0.0.0',port=port)    #这样用来监听所有的ip，团队调试用

 * Running on http://0.0.0.0:6006/ (Press CTRL+C to quit)


running at http://10.0.24.31:6006


10.0.24.28 - - [13/Jun/2018 21:00:19] "GET / HTTP/1.1" 200 -
10.0.24.28 - - [13/Jun/2018 21:00:23] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.24.28 - - [13/Jun/2018 21:00:23] "POST /predict HTTP/1.1" 200 -


query 123 123
result data get
saving to mongo db successfully!


10.0.24.28 - - [13/Jun/2018 21:00:24] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.24.28 - - [13/Jun/2018 21:00:24] "POST /predict HTTP/1.1" 200 -


...

query 123
result data get
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "202cb962ac59075b964b07152d234b70" }
重复查询同一句话,存储时使用相同的_id_ 202cb962ac59075b964b07152d234b70


10.0.24.28 - - [13/Jun/2018 21:00:24] "POST /dataFromAjax_post HTTP/1.1" 404 -
10.0.24.28 - - [13/Jun/2018 21:00:24] "POST /predict HTTP/1.1" 200 -


query 123
result data get
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "202cb962ac59075b964b07152d234b70" }
重复查询同一句话,存储时使用相同的_id_ 202cb962ac59075b964b07152d234b70


[2018-06-13 21:00:27,218] ERROR in app: Exception on /newConversation [POST]
Traceback (most recent call last):
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1982, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1614, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1517, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/_compat.py", line 33, in reraise
    raise value
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1612, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/kai/anaconda3/lib/python3.6/site-packages/flask/app.py", line 1598, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-7-dab40ce0ed61>", lin

query 123
result data get
E11000 duplicate key error collection: flask_database.query index: _id_ dup key: { : "202cb962ac59075b964b07152d234b70" }
重复查询同一句话,存储时使用相同的_id_ 202cb962ac59075b964b07152d234b70
